# Inicio del ETL sobre el archivo csv de juegos de steam

En esta parte de la investigacion, crearemos un nuevo csv el cual estara lo mas limpio posible y con columnas claves que nos serviran para despues. Aqui se explicaran los procesos y sesgos que ocuparemos para limpiar el csv de los juegos.

**Estaremos ocupando informacion sobre la API de "https://www.kaggle.com/datasets/artermiloff/steam-games-dataset". La cual contiene mas de 90.000 juegos.**

**En este nootebok analizaremos de forma superficial el comportamiento de los juegos originales y sus secuelas.**
**Viendo si efectivamente es normal que la secuela fracase, y si no es asi, ver como se comportan generalmente**
### Para que sea mas veridico, analizaremos solo juegos que tengan mas de 1000 reseñanas, para que estos ya sean catalogados como juegos con relevancia
**Todos los juegos de esta data provienen de Steam y las reseñas son de los mismo jugadores.**
***

## Librerias a ocupar
***

In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import json
import re
import seaborn as sns
from pathlib import Path


### Abrir el archivo
***

In [72]:
with open("data/games_march2025_cleaned.csv", "r", encoding='utf-8') as archivo:
    data = pd.read_csv(archivo)

### Informacion basica de la data
Mas abajo se puede apreciar las distintas columnas y cuantas filas. Destacando las columnas de:

`name`
`recommendations`
`positive`
`negative`
`pct_pos_total`
`num_reviews_total`
`pct_pos_recent`
`num_reviews_recent`
`release_date`

Estas columnas seran mas importantes para mas adelante por que nos indican de una forma rapida si el juego es mejor que su secuela o viceversa
***

In [73]:
data.head()

,appid,name,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,reviews,...,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,discount,peak_ccu,tags,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent
0,730,Counter-Strike 2,2012-08-21,0,0.00,1,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",NaN,...,879,5174,350,0,1212356,"{'FPS': 90857, 'Shooter': 65397, 'Multiplayer'...",86,8632939,82,96473
1,578080,PUBG: BATTLEGROUNDS,2017-12-21,0,0.00,0,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...","LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...",Play PUBG: BATTLEGROUNDS for free. Land on str...,NaN,...,0,0,0,0,616738,"{'Survival': 14838, 'Shooter': 12727, 'Battle ...",59,2513842,68,16720
2,570,Dota 2,2013-07-09,0,0.00,2,"The most-played game on Steam. Every day, mill...","The most-played game on Steam. Every day, mill...","Every day, millions of players worldwide enter...",“A modern multiplayer masterpiece.” 9.5/10 – D...,...,1536,898,892,0,555977,"{'Free to Play': 59933, 'MOBA': 20158, 'Multip...",81,2452595,80,29366
3,271590,Grand Theft Auto V Legacy,2015-04-13,17,0.00,0,"When a young street hustler, a retired bank ro...","When a young street hustler, a retired bank ro...",Grand Theft Auto V for PC offers players the o...,NaN,...,771,7101,74,0,117698,"{'Open World': 32644, 'Action': 23539, 'Multip...",87,1803832,92,17517
4,359550,Tom Clancy's Rainbow Six® Siege,2015-12-01,17,3.99,9,Edition Comparison Ultimate Edition The Tom Cl...,“One of the best first-person shooters ever ma...,"Tom Clancy's Rainbow Six® Siege is an elite, t...",NaN,...,682,2434,306,80,89916,"{'FPS': 9831, 'PvP': 9162, 'e-sports': 9072, '...",84,1168020,76,12608


In [74]:
data.columns

Index(['appid', 'name', 'release_date', 'required_age', 'price', 'dlc_count',
       'detailed_description', 'about_the_game', 'short_description',
       'reviews', 'header_image', 'website', 'support_url', 'support_email',
       'windows', 'mac', 'linux', 'metacritic_score', 'metacritic_url',
       'achievements', 'recommendations', 'notes', 'supported_languages',
       'full_audio_languages', 'packages', 'developers', 'publishers',
       'categories', 'genres', 'screenshots', 'movies', 'user_score',
       'score_rank', 'positive', 'negative', 'estimated_owners',
       'average_playtime_forever', 'average_playtime_2weeks',
       'median_playtime_forever', 'median_playtime_2weeks', 'discount',
       'peak_ccu', 'tags', 'pct_pos_total', 'num_reviews_total',
       'pct_pos_recent', 'num_reviews_recent'],
      dtype='object')

In [75]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89618 entries, 0 to 89617
Data columns (total 47 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   appid                     89618 non-null  int64  
 1   name                      89618 non-null  object 
 2   release_date              89618 non-null  object 
 3   required_age              89618 non-null  int64  
 4   price                     89618 non-null  float64
 5   dlc_count                 89618 non-null  int64  
 6   detailed_description      89421 non-null  object 
 7   about_the_game            89398 non-null  object 
 8   short_description         89498 non-null  object 
 9   reviews                   10401 non-null  object 
 10  header_image              89618 non-null  object 
 11  website                   41114 non-null  object 
 12  support_url               44110 non-null  object 
 13  support_email             78798 non-null  object 
 14  window

## Proceso ed limpieza de datos
**Con las columnas de antes destacadas, las dejaramos en el Dataframe final para analizar el comportamiento de las secuelas y como les va generalmente**
***

In [76]:
# Eliminar algunas columnas para limpiar
# Elimino todas las columnas para quedarme solo con las cosas positivas
columnas_eliminar = ["detailed_description", "about_the_game", "short_description",
                      "reviews", "header_image", "website", "support_url", "support_email",
                      "metacritic_url", "achievements", "notes", "screenshots", "movies",
                      "tags", 'required_age', 'price', 'dlc_count',
                      'windows', 'mac', 'linux','supported_languages',
                      'full_audio_languages', 'packages', 'developers', 'publishers',
                      'categories', 'genres', 'estimated_owners',
                      'average_playtime_forever', 'average_playtime_2weeks',
                      'median_playtime_forever', 'median_playtime_2weeks', 'discount',
                      'peak_ccu', 'tags', "user_score", "score_rank", "metacritic_score",
                      "num_reviews_recent", "pct_pos_recent", "recommendations"
                      ]

df_reseñas = data.drop(columns=columnas_eliminar)
df_reseñas.head()

,appid,name,release_date,positive,negative,pct_pos_total,num_reviews_total
0,730,Counter-Strike 2,2012-08-21,7480813,1135108,86,8632939
1,578080,PUBG: BATTLEGROUNDS,2017-12-21,1487960,1024436,59,2513842
2,570,Dota 2,2013-07-09,1998462,451338,81,2452595
3,271590,Grand Theft Auto V Legacy,2015-04-13,1719950,250012,87,1803832
4,359550,Tom Clancy's Rainbow Six® Siege,2015-12-01,1152763,218446,84,1168020


## Limpieza de las columnas y datos atopicos

**Dado que las escala de metacritic_score no siempre esta actualizada o no tiene los datos lo voy a eliminar**

**- Asique me voy a quedar con solo las reseñas de steam que son solo usuarios**
***

In [77]:
# Mucha limpieza

# Les cambio el nombre a las columnas para mayor comodidad
df_reseñas_limpio = df_reseñas.rename(columns={"appid": "id",
                                              'name': 'Nombre',
                                              "positive" : "Reseñas_positivas",
                                              "negative" : "Reseñas_negativas",
                                              "num_reviews_total" : "Nun_total_de_reseñas",
                                              "release_date" : "Fecha_de_lanzamiento",
                                              "pct_pos_total" : "Porcentaje_positivo_total"}
                                              )
                   
df_reseñas_limpio['Fecha_de_lanzamiento'] = pd.to_datetime(df_reseñas_limpio['Fecha_de_lanzamiento'], errors='coerce')

# Elimino las columnas que deben ser claves
df_reseñas_limpio = df_reseñas_limpio.dropna(subset=["Nun_total_de_reseñas", "Nombre",
                                                    'Fecha_de_lanzamiento', 'Reseñas_positivas',
                                                    "Reseñas_negativas",
                                                    ])

# Elimino filas en las que hayan datos raros y minimo 1000 reseñas en total

# Definimos las condiciones para eliminar los datos que no queremos
condicion_total = df_reseñas_limpio['Nun_total_de_reseñas'] >= 1000

condicion_positivas = df_reseñas_limpio['Reseñas_positivas'] >= 0

condicion_negativas = df_reseñas_limpio['Reseñas_negativas'] >= 0

# Creamos un nuevo df con las nuevas condiciones
df_filtrado = df_reseñas_limpio[condicion_total & condicion_positivas & condicion_negativas]


# Hacemos que el df esta en orden ascendente por el año de lanzamiento
df_filtrado = df_filtrado.sort_values(by='Fecha_de_lanzamiento', ascending=True)

df_filtrado["Nun_total_de_reseñas"].describe()




count    6.232000e+03
mean     1.801543e+04
std      1.332200e+05
min      1.000000e+03
25%      1.637750e+03
50%      3.081500e+03
75%      8.741250e+03
max      8.632939e+06
Name: Nun_total_de_reseñas, dtype: float64

In [78]:
df_filtrado.head()

,id,Nombre,Fecha_de_lanzamiento,Reseñas_positivas,Reseñas_negativas,Porcentaje_positivo_total,Nun_total_de_reseñas
1369,232770,POSTAL,1997-11-14,9487,961,90,10472
1918,20,Team Fortress Classic,1999-04-01,7500,1121,86,6482
1312,65540,Gothic 1,2001-03-15,14162,1209,91,11058
1640,40990,Mafia,2002-08-28,13116,1862,90,8150
131,240,Counter-Strike: Source,2004-11-01,172801,6697,96,124438


**La columna de total de reseñas no la ocupare ahora pero sirve para discriminar el total de reseñas**
***
## Avanzamos en la agrupacion los juegos con sus secuelas y ver si estos son bien resividos mediante las reseñas
En esta parte de la investigacion, es muy dificil separar juegos por sus secuelas, asque abran juegos que no esten con sus secuelas dado que cambian mucho su nombre. **Asique no es exacto este metodo y tiene margen de error**
***

In [79]:
def obtener_nombre_franquicia(nombre):
    if not isinstance(nombre, str):
        return ""
    
    # Sacar signos
    s = nombre.lower().strip()
    s = re.sub(r'[™®©]', '', s)
    
    # Eliminar signos
    s = re.split(r'[:—]| - ', s)[0]
    
    # En esta parte ignora muchas de las palabras que no sirven
    palabras_clave = r'\b(remastered|definitive|edition|goty|game of the year|directors cut|enhanced|biohazard|village|vr|hd|remake)\b'
    s = re.sub(palabras_clave, '', s)
    
    # quitar numeros romanos
    s = re.sub(r'\b(ii|iii|iv|v|vi|vii|viii|ix|x|xi|xii)\b$', '', s)
    
    # Quitar numeros
    s = re.sub(r'\b\d+$', '', s.strip())
    
    s = re.sub(r'[^\w\s]', '', s).strip()
    
    return s

# Creo la columna que va hacer la base para crear las franquicias
df_filtrado['franquicia_id'] = df_filtrado['Nombre'].apply(obtener_nombre_franquicia)

# Hago que la fecha sea datetima para sacar calculos mas faciles
df_filtrado['Fecha_de_lanzamiento'] = pd.to_datetime(df_filtrado['Fecha_de_lanzamiento'], errors='coerce')
df_filtrado['release_year'] = df_filtrado['Fecha_de_lanzamiento'].dt.year
df_filtrado.head()

,id,Nombre,Fecha_de_lanzamiento,Reseñas_positivas,Reseñas_negativas,Porcentaje_positivo_total,Nun_total_de_reseñas,franquicia_id,release_year
1369,232770,POSTAL,1997-11-14,9487,961,90,10472,postal,1997
1918,20,Team Fortress Classic,1999-04-01,7500,1121,86,6482,team fortress classic,1999
1312,65540,Gothic 1,2001-03-15,14162,1209,91,11058,gothic,2001
1640,40990,Mafia,2002-08-28,13116,1862,90,8150,mafia,2002
131,240,Counter-Strike: Source,2004-11-01,172801,6697,96,124438,counterstrike,2004


***
## Creamos el data frame final que rescate todas las secuelas posibles

Ya que este metodo que utilizo hace que no tenga un 100% de efectividad al capturar las franquicias y las secuelas. Capturo todas las posibles y trabajo con ellas

In [80]:
conteo_franquicias = df_filtrado['franquicia_id'].value_counts()
franquicias_validas = conteo_franquicias[conteo_franquicias > 1].index
df_filtrado = df_filtrado[(df_filtrado['franquicia_id'].isin(franquicias_validas)) & (df_filtrado['franquicia_id'] != "")].copy()

# Ordenar por fecha de lanzamiento y franquicia
df_filtrado = df_filtrado.sort_values(by=['franquicia_id', 'Fecha_de_lanzamiento'])

# Lista para guardar los diccionarios de las secuelas
secuelas_data = []

# Iteramos sobre cada franquicia
for franquicia, grupo in df_filtrado.groupby('franquicia_id'):
    # Este es el juego original
    juego_original = grupo.iloc[0]
    
    # Si no tiene fecha se quita
    if pd.isna(juego_original['Fecha_de_lanzamiento']):
        continue
        
    # Esto para seguir con las secuelas
    secuelas = grupo.iloc[1:]
    
    for _, juego_secuela in secuelas.iterrows():
        if pd.isna(juego_secuela['Fecha_de_lanzamiento']):
            continue
            
        anios_diferencia = juego_secuela['release_year'] - juego_original['release_year'] # Columna pa mejor informacion
        
        # Esto nos quita la division por cero
        total_reviews_orig = juego_original['Reseñas_positivas'] + juego_original['Reseñas_negativas']
        total_reviews_secuela = juego_secuela['Reseñas_positivas'] + juego_secuela['Reseñas_negativas']
        
        ratio_popularidad = 0
        if total_reviews_orig > 0:
            ratio_popularidad = (total_reviews_secuela / total_reviews_orig) * 100
        
        # Esta ultima parte es para retornar la fila con todos los datos importantes que necesitamos
        data_row = {
            'Franquicia_Base': franquicia,
            # Datos Secuela
            'Secuela_Nombre': juego_secuela['Nombre'],
            'Secuela_Año': int(juego_secuela['release_year']),
            'Secuela_Reseñas_Positivas': juego_secuela['Reseñas_positivas'],
            'Secuela_Reseñas_Negativas': juego_secuela['Reseñas_negativas'],
            'Secuela_Pct_Positivo': juego_secuela['Porcentaje_positivo_total'],
            
            # Datos Original
            'Original_Nombre': juego_original['Nombre'],
            'Original_Año': int(juego_original['release_year']),
            'Original_Reseñas_Positivas': juego_original['Reseñas_positivas'],
            'Original_Reseñas_Negativas': juego_original['Reseñas_negativas'],
            'Original_Pct_Positivo': juego_original['Porcentaje_positivo_total'],
            
            # Métricas Comparativas
            'Años_Diferencia': anios_diferencia,
            'Cambio_Pct_Calidad': juego_secuela['Porcentaje_positivo_total'] - juego_original['Porcentaje_positivo_total'], # Con esto se ve si subio o bajo la calidad
            'Ratio_Popularidad_vs_Original': round(ratio_popularidad, 2) # Si es mayor a 100% entonces la secuela "mejoro" y es mas popular
        }
        # Retornamos la fila que despues se combertira en un df
        secuelas_data.append(data_row)

# Con las filas de la lista. Creamos el df
df_final = pd.DataFrame(secuelas_data)

In [81]:
df_final.head(10)

,Franquicia_Base,Secuela_Nombre,Secuela_Año,Secuela_Reseñas_Positivas,Secuela_Reseñas_Negativas,Secuela_Pct_Positivo,Original_Nombre,Original_Año,Original_Reseñas_Positivas,Original_Reseñas_Negativas,Original_Pct_Positivo,Años_Diferencia,Cambio_Pct_Calidad,Ratio_Popularidad_vs_Original
0,12 labours of hercules,12 Labours of Hercules II: The Cretan Bull,2015,1867,208,90,12 Labours of Hercules,2015,2814,192,93,0,-3,69.03
1,9nine,9-nine-:Episode 2,2019,1491,23,98,9-nine-:Episode 1,2019,1461,73,95,0,3,98.70
2,9nine,9-nine-:Episode 4,2021,1089,20,98,9-nine-:Episode 1,2019,1461,73,95,2,3,72.29
3,a plague tale,A Plague Tale: Requiem,2022,17807,1721,91,A Plague Tale: Innocence,2019,58010,4066,93,3,-2,31.46
4,a space for the unbound,A Space for the Unbound,2023,5674,140,97,A Space For The Unbound - Prologue,2020,1276,24,98,3,-1,447.23
5,a total war saga,A Total War Saga: TROY,2021,2303,914,71,A Total War Saga: THRONES OF BRITANNIA,2018,9829,4191,69,3,2,22.95
6,above snakes,Above Snakes,2023,1258,393,75,Above Snakes: Prologue,2023,0,0,90,0,-15,0.00
7,age of empires,Age of Empires II: Definitive Edition,2019,150058,8044,94,Age of Empires: Definitive Edition,2019,11384,2182,83,0,11,1165.43
8,age of empires,Age of Empires III: Definitive Edition,2020,39670,7287,84,Age of Empires: Definitive Edition,2019,11384,2182,83,1,1,346.14
9,age of empires,Age of Empires IV: Anniversary Edition,2021,66268,10754,86,Age of Empires: Definitive Edition,2019,11384,2182,83,2,3,567.76


In [82]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 805 entries, 0 to 804
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Franquicia_Base                805 non-null    object 
 1   Secuela_Nombre                 805 non-null    object 
 2   Secuela_Año                    805 non-null    int64  
 3   Secuela_Reseñas_Positivas      805 non-null    int64  
 4   Secuela_Reseñas_Negativas      805 non-null    int64  
 5   Secuela_Pct_Positivo           805 non-null    int64  
 6   Original_Nombre                805 non-null    object 
 7   Original_Año                   805 non-null    int64  
 8   Original_Reseñas_Positivas     805 non-null    int64  
 9   Original_Reseñas_Negativas     805 non-null    int64  
 10  Original_Pct_Positivo          805 non-null    int64  
 11  Años_Diferencia                805 non-null    int32  
 12  Cambio_Pct_Calidad             805 non-null    int

### Vemos cual es el dato outlier que hace que los graficos de dispersen
Vemos que Counter-Strike 2 tubo mucha mas fama que el juego original, pero esto es una acepcion a la norma, asique por fines de estudio eliminare este dato para hacer una grafica mas representativa
***

In [83]:
df_outlier = df_final.sort_values(by='Secuela_Reseñas_Positivas', ascending=False).head(1)

df_outlier.head()

,Franquicia_Base,Secuela_Nombre,Secuela_Año,Secuela_Reseñas_Positivas,Secuela_Reseñas_Negativas,Secuela_Pct_Positivo,Original_Nombre,Original_Año,Original_Reseñas_Positivas,Original_Reseñas_Negativas,Original_Pct_Positivo,Años_Diferencia,Cambio_Pct_Calidad,Ratio_Popularidad_vs_Original
122,counterstrike,Counter-Strike 2,2012,7480813,1135108,86,Counter-Strike: Source,2004,172801,6697,96,8,-10,4800.01


In [84]:
# Ya que encontramos el juego que hace que los graficos se vean mal. Lo eliminamos
indice_outlier = df_final['Secuela_Reseñas_Positivas'].idxmax()

df_limpio_sin_outlier = df_final.drop(indice_outlier)

In [85]:
df_limpio_sin_outlier.head()

,Franquicia_Base,Secuela_Nombre,Secuela_Año,Secuela_Reseñas_Positivas,Secuela_Reseñas_Negativas,Secuela_Pct_Positivo,Original_Nombre,Original_Año,Original_Reseñas_Positivas,Original_Reseñas_Negativas,Original_Pct_Positivo,Años_Diferencia,Cambio_Pct_Calidad,Ratio_Popularidad_vs_Original
0,12 labours of hercules,12 Labours of Hercules II: The Cretan Bull,2015,1867,208,90,12 Labours of Hercules,2015,2814,192,93,0,-3,69.03
1,9nine,9-nine-:Episode 2,2019,1491,23,98,9-nine-:Episode 1,2019,1461,73,95,0,3,98.70
2,9nine,9-nine-:Episode 4,2021,1089,20,98,9-nine-:Episode 1,2019,1461,73,95,2,3,72.29
3,a plague tale,A Plague Tale: Requiem,2022,17807,1721,91,A Plague Tale: Innocence,2019,58010,4066,93,3,-2,31.46
4,a space for the unbound,A Space for the Unbound,2023,5674,140,97,A Space For The Unbound - Prologue,2020,1276,24,98,3,-1,447.23


In [86]:
df_limpio_sin_outlier.info()

<class 'pandas.core.frame.DataFrame'>
Index: 804 entries, 0 to 804
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Franquicia_Base                804 non-null    object 
 1   Secuela_Nombre                 804 non-null    object 
 2   Secuela_Año                    804 non-null    int64  
 3   Secuela_Reseñas_Positivas      804 non-null    int64  
 4   Secuela_Reseñas_Negativas      804 non-null    int64  
 5   Secuela_Pct_Positivo           804 non-null    int64  
 6   Original_Nombre                804 non-null    object 
 7   Original_Año                   804 non-null    int64  
 8   Original_Reseñas_Positivas     804 non-null    int64  
 9   Original_Reseñas_Negativas     804 non-null    int64  
 10  Original_Pct_Positivo          804 non-null    int64  
 11  Años_Diferencia                804 non-null    int32  
 12  Cambio_Pct_Calidad             804 non-null    int64  


***
## Ya hecho el ETL y con datas sufucientemente limpios a nuestro juicio, lo guardamos en un nuevo df

In [87]:
# Creo un csv con el data freme final limpio

carpeta = Path('data')  # La donde lo voy a guardar
archivo = carpeta / 'df_limpio_sin_outlier_resenas.csv' # El nombre del csv

# 4. Guardar
df_limpio_sin_outlier.to_csv(archivo, index=False) 

print(f"Guardado exitosamente en: {archivo}")


Guardado exitosamente en: data\df_limpio_sin_outlier_resenas.csv
